![test](files/tables/header.png)

####CMM705 Big Data Programming Coursework (Sep 2019)

## Airbnb Singapore Machine Learning Model

Binary classification using pyspark and mllib libiries to predict neigbourhood group based on the latitute and longitde features. All the string columns has converted into vectors

#### 01. Retrieving the data from csv

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('/FileStore/tables/listingsoriginal.csv', header = True, inferSchema = True)
df.printSchema()

totalCount = df.count()

root
-- id: string (nullable = true)
-- name: string (nullable = true)
-- host_id: string (nullable = true)
-- host_name: string (nullable = true)
-- neighbourhood_group: string (nullable = true)
-- neighbourhood: string (nullable = true)
-- latitude: double (nullable = true)
-- longitude: string (nullable = true)
-- room_type: string (nullable = true)
-- price: integer (nullable = true)
-- minimum_nights: integer (nullable = true)
-- number_of_reviews: string (nullable = true)
-- last_review: string (nullable = true)
-- reviews_per_month: double (nullable = true)
-- calculated_host_listings_count: integer (nullable = true)
-- availability_365: integer (nullable = true)

**Input variables:** Lat, Long
**Output variable:** Neigbourhood group

In [ ]:
# select input variables and output variables only
df = df.select('latitude','longitude', 'neighbourhood_group')
cols = df.columns
df.printSchema()

Error: Error: Jupyter cannot be started. Error attempting to locate jupyter: Error: Module 'notebook' not installed.

In [8]:
# convert str to double
df = df.withColumn('latitude',df['latitude'].cast("double")).withColumn('longitude',df['longitude'].cast("double"))

In [9]:
#drop null values
df = df.dropna()
nullValuesCount = totalCount - df.count()
nullValuesCount

Out[10]: 26

In [10]:
display(df)

In [11]:
grouping_data = df.groupby('neighbourhood_group').count()
display(grouping_data)

neighbourhood_group,count
West Region,539
Central Region,6301
North Region,203
East Region,508
North-East Region,344


#### 02. Preparing Data for Machine Learning

In [13]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

stages = []

label_stringIdx = StringIndexer(inputCol = 'neighbourhood_group', outputCol = 'label')
stages += [label_stringIdx]

assemblerInputs = ['latitude', 'longitude']
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]


In [14]:
# Pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
-- label: double (nullable = false)
-- features: vector (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- neighbourhood_group: string (nullable = true)

In [15]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
label,4,0,4,2,2
features,"[1.44255, 103.7958]","[1.33235, 103.78521]","[1.44246, 103.79667]","[1.34541, 103.95712]","[1.34567, 103.95963]"
latitude,1.44255,1.33235,1.44246,1.34541,1.34567
longitude,103.796,103.785,103.797,103.957,103.96
neighbourhood_group,North Region,Central Region,North Region,East Region,East Region


In [16]:
# split data for testing and training

train, test = df.randomSplit([0.8, 0.2], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 6310
Test Dataset Count: 1585

#### 03. Use the Decision Tree Classifier

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select('latitude', 'longitude', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+--------+---------+-----+--------------------+----------+--------------------+
latitude|longitude|label| rawPrediction|prediction| probability|
+--------+---------+-----+--------------------+----------+--------------------+
 1.25284|103.82225| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.26624|103.81097| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.26675|103.81219| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.26814|103.81203| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.26863| 103.8239| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.26983|103.81331| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.27173|103.82232| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.27234|103.83224| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.27237|103.83233| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
 1.27239|103.83419| 0.0|[4935.0,19.0,26.0...| 0.0|[0.99096385542168...|
+--------+---------+-----+--------------------+----------+--------------------+
only showing top 10 rows

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.3278602373507068

#### 03. Use the Logistic Regression Model

In [21]:
from pyspark.ml.classification import LogisticRegression

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(train)

In [22]:
# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

In [23]:
# Make predictions on the test set

predictions = mlrModel.transform(test)
predictions.select('latitude', 'longitude', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

In [24]:
#Evaluate our Logistic Regression model.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))